In [ ]:
### install dependencies:

# might need to install opencv
!pip3 install Cython cupy-cuda112 cupy-cuda102   # Probably only need one of these but it works so I'm gonna leave it for now.
!pip3 install pyyaml==5.1

# Torch 1.8.1 does not work despite being the latest stable release. As such, use 1.7. 
# Cuda version on MAGEOHub is currently 11.3 - cu110 all works fine.

# To be tried once the bug with torch 1.8 has been fixed 
#!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!python -m pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html

In [ ]:
# Import torch and check versions and cuda availability, and resources
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
!nvcc --version
!nvidia-smi

from IPython.display import display, clear_output

In [ ]:
# Some basic imports:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# tensorboard?
#%load_ext tensorboard
#%tensorboard --logdir output

# necessary libraries
import pandas as pd
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import json
import glob

#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
# Make sure the inevitable error messages are useful.
CUDA_LAUNCH_BLOCKING="1"

In [ ]:
# Setup to predict on new images, here setting up for the trees_test dataset, but can also use this setup
# for predicting on individual images as seen 2 cells down

from detectron2.utils.visualizer import ColorMode
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

### path to the saved pre-trained model weights
cfg.MODEL.WEIGHTS = '/home/jovyan/detectron2_live/train_test/output/model_final.pth'

# set confidence threshold at which we predict
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.15

predictor = DefaultPredictor(cfg)

In [ ]:
# make and visualise predictions, on a particular file 

# give the image that we are trying to predict on

im = cv2.imread("/home/jovyan/lustre_scratch/sepilok_data/spare_pngs/tile_603000_646900.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1], scale=1.5, instance_mode=ColorMode.IMAGE_BW)   # remove the colors of unsegmented pixels
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
image = cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
display(Image.fromarray(image))

In [ ]:
import rasterio
from rasterio.transform import from_origin

In [ ]:
filepath = "/home/jovyan/lustre_scratch/sepilok_data/spare_pngs/tile_603000_646900.png"
im = cv2.imread(filepath)
outputs = predictor(im)

mask_array = outputs['instances'].pred_masks.cpu().numpy()
    
# get confidence scores too 
# mask_array_scores = outputs['instances']['scores'].pred_masks.cpu().numpy()
    
num_instances = mask_array.shape[0]
#print(num_instances)
mask_array_instance = []
# create an array of the right shape
output = np.zeros_like(mask_array) 
#print('output',output.shape)

mask_array_instance.append(mask_array)
output = np.where(mask_array_instance[0] == True, 255, output)
#print(output)
#print(mask_array_instance)
fresh_output = output.astype(np.float)
#print(fresh_output.shape)
x_scaling = 140/fresh_output.shape[1]
y_scaling = 140/fresh_output.shape[2]
# this is an affine transform. This needs to be altered significantly.
transform = from_origin(int(filepath[-17:-11])-20, int(filepath[-10:-4])+120, y_scaling, x_scaling)

new_dataset = rasterio.open('/home/jovyan/lustre_scratch/10_predicted_polygons_'+filepath[-17:-4]+'.tif', 'w', driver='GTiff',
                             height = fresh_output.shape[1], width = fresh_output.shape[2], count = fresh_output.shape[0],
                             dtype=str(fresh_output.dtype),
                             crs='+proj=utm +zone=22 +datum=WGS84 +units=m +no_defs',  
                             transform=transform)

new_dataset.write(fresh_output)
new_dataset.close()


In [ ]:
### THIS WORKS BRILLIANTLY
### It reads in a stack raster of all the predictions, and outputs n shapefiles, where n 
### is the number of predictions for that particular image. 
### We can then merge all these predictions...need to do this in python 

#### extract confidence from the predictions to include as a column in the shapefile

# Another way to polygonise all the bands of my raster

import numpy as np
import fiona
import rasterio
import rasterio.features
from shapely.geometry import shape, mapping
from shapely.geometry.multipolygon import MultiPolygon
#from shapely.geometry import Polygon


# Read input band with Rasterio
tiff_filepath = '/home/jovyan/lustre_scratch/10_predicted_polygons_603000_646900.tif'
with rasterio.open(filepath) as src:
    shp_schema = {'geometry': 'MultiPolygon','properties': {'pixelvalue': 'int'}}
    
    crs = src.crs
    for i in range(src.count):
        src_band = src.read(i+1)
        src_band = np.float32(src_band)
        #score = array[i]
        #print(src_band.dtype)
        # Keep track of unique pixel values in the input band
        unique_values = np.unique(src_band)
        # Polygonize with Rasterio. `shapes()` returns an iterable
        # of (geom, value) as tuples
        shapes = list(rasterio.features.shapes(src_band, transform=src.transform))


# simplify here

# Get a list of all polygons for a given pixel value
# and create a MultiPolygon geometry with shapely.
# Then write the record to an output shapefile with fiona.
# We make use of the `shape()` and `mapping()` functions from
# shapely to translate between the GeoJSON-like dict format
# and the shapely geometry type.
##### Don't forget to add this folder here!
### change ' w' to 'a' to append....

        with fiona.open('/home/jovyan/lustre_scratch/folder/10_predicted_polygons_'+filepath[-17:-4]+'_'+str(i)+'.shp', 'w', 'ESRI Shapefile', shp_schema, crs) as shp:
            for pixel_value in unique_values:
                polygons = [shape(geom) for geom, value in shapes
                            if value == pixel_value]
                multipolygon = MultiPolygon(polygons)
                shp.write({
                    'geometry': mapping(multipolygon),
                    'properties': {'pixelvalue': int(pixel_value)}   # 'score': {mask_array_score[i]}
                        })

In [ ]:
### zip up the folder to download
import shutil

shutil.make_archive("/home/jovyan/lustre_scratch/folder/zipped_set_shapefiles", "zip", "/home/jovyan/lustre_scratch/paracou_data/folder")

#zip -r /home/jovyan/lustre_scratch/sepilok_data/all_sepilok_preds_shapes.zip /home/jovyan/lustre_scratch/sepilok_data/all_sepilok_preds_shapes